In [81]:
import csv 
import gzip 
from sklearn.model_selection import cross_val_score
from sklearn import linear_model


# reading in the data to get Field Names
data = []
with open('Mortality_05_UT.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    fieldNames = []
    for row in spamreader:
        fieldNames.append(row)
        break

In [82]:
fieldNames = fieldNames[0]

In [83]:
# create List of dictionaries for each record in the dataset
with open('Mortality_05_UT.csv') as csvfile:
    data = []
    reader = csv.DictReader(csvfile,fieldnames=fieldNames)
    count = 0 
    for row in reader:
        if count != 0:
            data.append(row)
        count +=1
        

In [84]:
# remove records that have an empty age field 
cleanData = []
for datum in data:
    toAppend = True
    if datum['age'] is '':
        toAppend = False
    for k in datum.keys():
        if datum[k] is '':
            datum[k] = -100
    if toAppend:
        cleanData.append(datum)

In [85]:
useless = ['m_id','client_m_id','hl_id','house_no','PSU_ID','m_serial_no','date_of_death','month_of_death','year_of_death','is_death_reg','place_of_death','is_death_certificate_received','serial_num_of_infant_mother','death_period','months_of_pregnancy']

In [86]:
useful_stuff = ['id','state','house_hold_no','district','rural','stratum_code',\
               'deceased_sex','treatment_source','order_of_birth','sex','age',\
               'religion','marital_status','currently_attending_school', \
               'reason_for_not_attending_school','highest_qualification','occupation_status'\
              'disability_status','regular_treatment','regular_treatment_source','chew','smoke'\
              'injury_treatment_type','alcohol','house_status','owner_status','drinking_water_source'\
              'is_water_filter','water_filtration','toilet_used','is_toilet_shared','household_have_electricity'\
              'lighting_source','cooking_fuel','no_of_dwelling_rooms','kitchen_availability','is_radio',\
              'is_televison','is_computer','is_telephone', 'is_washing_machine', 'is_refrigerator', \
              'is_sewing_machine', 'is_bicycle', 'is_scooter', 'is_car', 'is_tractor', 'is_water_pump', \
              'cart', 'land_possessed', 'iscoveredbyhealthscheme', 'injury_treatment_type'\
              'house_structure']

In [87]:
columns = set(['id','state','house_hold_no','district','rural','stratum_code',\
'private_parts','order_of_birth','age','religion','marital_status'\
,'currently_attending_school', 'reason_for_not_attending_school','regular_treatment','regular_treatment_source','chew','smoke'\
,'injury_treatment_type','alcohol','house_status','owner_status','drinking_water_source'\
,'water_filtration_category','toilet_used','is_toilet_shared','household_have_electricity'\
,'lighting_source','cooking_fuel','no_of_dwelling_rooms','kitchen_availability','is_radio',\
'is_television','is_computer','is_telephone', 'is_washing_machine', 'is_refrigerator','is_sewing_machine', 'is_bicycle', 'is_scooter', 'is_car', 'is_tractor', 'is_water_pump', \
'is_cart','land_possessed','iscoveredbyhealthscheme'\
,'house_structure'])

columns = list(columns)

In [88]:
from collections import defaultdict
consolidated_data = []
count = 0 
for d in cleanData:
    print count
    tempDict = defaultdict(int)
    sex = None 
    
    for c in columns:
        if c is 'private_parts':
            if d['sex'] != -100:
                sex = int(d['sex'])
            elif d['deceased_sex'] != -100:
                sex = int(d['deceased_sex'])
            tempDict['private_parts'] = sex 
        elif c is 'is_cart':
            tempDict[c] = int(d['cart'])
        else:
            print c,d[c]
            tempDict[c] = int(d[c])
    count +=1
        
        
            

0
chew 3
is_radio 1
is_television 1
is_water_pump 2
drinking_water_source 1
is_toilet_shared 2
reason_for_not_attending_school -100
is_tractor 2
order_of_birth -100
household_have_electricity 1
id

KeyError: 'id'

In [90]:
def feature(datum):
    feat = []
    for k in datum.keys():
        if k in useful_stuff:
            break
    return feat

In [91]:
sex_count = 0 
sex = 0 
for d in data:
    if d['sex'] is not '':
        sex = d['sex']
    elif d['deceased_sex'] is not '':
        sex = d['deceased_sex']
    if sex == -100:
        sex_count +=1 
        
print sex_count
len(data)


427


53503

In [92]:
type(data[200]['deceased_sex'])

str

In [96]:
columns.index('private_parts')

4

In [73]:
len([d for d in cleanData if d['id']] is not '')

KeyError: 'id'

In [75]:
data[1]

{'age': '28',
 'age_of_death_above_one_year': '67',
 'age_of_death_below_eleven_month': -100,
 'age_of_death_below_one_month': -100,
 'alcohol': '2',
 'as': -100,
 'building_no': -100,
 'cart': '4',
 'chew': '3',
 'client_hh_id': '12506',
 'client_hl_id': '6086',
 'client_m_id': -100,
 'cooking_fuel': '1',
 'currently_attending_school': -100,
 'currently_dead_or_out_migrated': '2',
 'date_of_birth': '10',
 'date_of_death': '6',
 'date_of_marriage': '19',
 'death_period': -100,
 'death_symptoms': -100,
 'deceased_sex': '1',
 'diagnosed_for': '0',
 'diagnosis_source': -100,
 'disability_status': '0',
 'district': '2',
 'drinking_water_source': '1',
 'factors_contributing_death': -100,
 'factors_contributing_death_2': -100,
 'father_serial_no': '1',
 'fid': '502217011101',
 'fidh': '50221701110102',
 'fidx': -100,
 'field38': -100,
 'healthscheme_1': -100,
 'healthscheme_2': -100,
 'hh_expall_status': '1',
 'hh_id': '14219',
 'hh_serial_no': '2',
 'highest_qualification': '4',
 'hl_expall

In [77]:
print type(data[0])

<type 'dict'>


In [80]:
print cleanData[0].keys()

['death_symptoms', 'is_television', 'drinking_water_source', 'deceased_sex', 'is_death_associated_with_pregnan', 'hl_id', 'is_refrigerator', 'stratum_code', 'social_group_code', 'date_of_birth', 'fid', 'place_of_death', 'is_car', 'is_scooter', 'relation_to_head', 'healthscheme_2', 'member_identity', 'date_of_death', 'usual_residance', 'lighting_source', 'death_period', 'month_of_marriage', 'marital_status', 'father_serial_no', 'field38', 'building_no', 'occupation_status', 'x', 'water_filteration', 'chew', 'is_radio', 'isdeadmigrated', 'serial_no', 'hh_id', 'sex', 'reason_for_not_attending_school', 'schedule_id', 'order_of_birth', 'year', 'alcohol', 'house_status', 'hl_expall_status', 'highest_qualification', 'religion', 'state', 'no_of_dwelling_rooms', 'sought_medical_care', 'housestatus', 'client_hh_id', 'house_hold_no', 'client_m_id', 'time_between_onset_of_complicati', 'cart', 'year_of_marriage', 'wt', 'is_sewing_machine', 'kitchen_availability', 'is_water_filter', 'is_bicycle', 'h